# Predicting melanoma in pytorch using Resnet18

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from collections import OrderedDict
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

#python packages
from PIL import Image
from tqdm import tqdm

import gc
import datetime
import copy
import matplotlib.pyplot as plt
import time
from skimage import io

#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

#torchvision
import torchvision
from torchvision import datasets, models, transforms
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

import random
import cv2
import warnings
import seaborn as sns

In [ ]:
#download the pretrained model
import torchvision.models as models
model = models.resnet18(pretrained = False)
model

#switch device to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load Data

In [ ]:
class AdvancedHairAugmentation:
    """
    Impose an image of a hair to the target image

    Args:
        hairs (int): maximum number of hairs to impose
        hairs_folder (str): path to the folder with hairs images
    """

    def __init__(self, hairs: int = 5, hairs_folder: str = ""):
        self.hairs = hairs
        self.hairs_folder = hairs_folder

    def __call__(self, img_path):
        """
        Args:
            img (PIL Image): Image to draw hairs on.

        Returns:
            PIL Image: Image with drawn hairs.
        """
        img = cv2.imread(img_path)
        n_hairs = random.randint(1, self.hairs) #choose a random number of hairs to add to image
        
        if not n_hairs:
            return img
        
        height, width, _ = img.shape  # target image width and height
        hair_images = [im for im in os.listdir(self.hairs_folder) if 'png' in im]
        
        for _ in range(n_hairs):
            hair = cv2.imread(os.path.join(self.hairs_folder, random.choice(hair_images)))
            
            #random flips & rotations of the hair strands
            hair = cv2.flip(hair, random.choice([-1, 0, 1]))
            hair = cv2.rotate(hair, random.choice([0, 1, 2]))

            h_height, h_width, _ = hair.shape  # hair image width and height
            roi_ho = random.randint(0, img.shape[0] - hair.shape[0])
            roi_wo = random.randint(0, img.shape[1] - hair.shape[1])
            roi = img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask
            img2gray = cv2.cvtColor(hair, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of hair in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of hair from hair image.
            hair_fg = cv2.bitwise_and(hair, hair, mask=mask)

            # Put hair in ROI and modify the target image
            dst = cv2.add(img_bg, hair_fg)

            img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)       
        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(hairs={self.hairs}, hairs_folder="{self.hairs_folder}")'

In [ ]:
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
import torchvision
class MultimodalDataset(Dataset):
    """
    Custom dataset definition
    """
    def __init__(self, csv_path, img_path, mode='train', transform=None):
        """
        """
        self.df = pd.read_csv(csv_path)
        self.img_path = img_path
        self.mode= mode
        self.transform = transform
        
            
    def __getitem__(self, index):
        """
        """
        img_name = self.df.iloc[index]["image_name"] + '.jpg'
        img_path = os.path.join(self.img_path, img_name)
        image = Image.open(img_path)
        
        dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor # ???
        
        if self.mode == 'train':
            if self.df.iloc[index]["augmented"]==1: #adds hair augmentation to malig images
                image = AdvancedHairAugmentation(hairs_folder="../input/melanoma-hairs")(img_path)
                image = Image.fromarray(image, 'RGB')
            elif self.df.iloc[index]["augmented"]==2: #adds hair augmentation to malig images again
                image = AdvancedHairAugmentation(hairs_folder="../input/melanoma-hairs")(img_path)
                image = Image.fromarray(image, 'RGB')
            else:  
                image = image.convert("RGB")
                
            image = np.asarray(image)
            if self.transform is not None:
                image = self.transform(image)
            labels = self.df.iloc[index]["target"]
            return image, labels
            
        elif self.mode == 'val':
            image = np.asarray(image)
            if self.transform is not None:
                image = self.transform(image)
            labels = self.df.iloc[index]["target"]
            return image, labels
        
        else: #when self.mode=='test'
            image = np.asarray(image)
            if self.transform is not None:
                image = self.transform(image)
            return image, self.df.iloc[index]["image_name"]

    def __len__(self):
        return len(self.df)


In [ ]:
def get_dataloaders(input_size, batch_size, augment=False, shuffle = True):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'test': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    
    data_subsets = {x: MultimodalDataset(csv_path="../input/melanoma/"+x+".csv", 
                                         img_path = image_path_dict[x],
                                         mode = x,
                                         transform=data_transforms[x]) for x in data_transforms.keys()}
    # Create training and validation dataloaders
    # Never shuffle the test set
    dataloaders_dict = {x: DataLoader(data_subsets[x], batch_size=batch_size, shuffle=False if x != 'train' else shuffle, num_workers=4) for x in data_transforms.keys()}
    return dataloaders_dict

In [ ]:
image_path_dict = {'train': "../input/siim-isic-melanoma-classification/jpeg/train",
                  'val': "../input/siim-isic-melanoma-classification/jpeg/train" ,
                  'test': "../input/siim-isic-melanoma-classification/jpeg/test"}

In [ ]:
dataloaders = get_dataloaders(input_size=224, batch_size=64, shuffle=True)

In [ ]:
train_loader = dataloaders['train']
val_loader = dataloaders['val']

In [ ]:
#Classifier architecture to put on top of resnet18
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512,100)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(100,2)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = fc

In [ ]:
#shifting model to gpu
model.to(device)
model

In [ ]:
num_classes = 2

#Different model parameters to play around with
num_epochs = 20
batch_size = 32
learning_rate = 0.01

In [ ]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(train_loader)
    
    #take out the following code if just starting to train
    #checkpoint = torch.load('PREVIOUS MODEL PATH')  for ex: torch.load('../input/trained-model/model_21.pth')
    #model.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    #epoch_before = checkpoint['epoch']
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(train_loader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            outputs = torch.squeeze(outputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        #'epoch' = epoch if just starting to train
        #'epoch' = epoch+epoch_before+1 afterwards
        model_file = { 'epoch': epoch+epoch_before+1,
                      'state_dict': model.state_dict(),
                      'optimizer' : optimizer.state_dict()}

        torch.save(model_file, "model" + str(epoch+epoch_before+1) + '.pth')  
        #str(epoch) if just starting to train 
        #str(epoch+epoch_before+1) afterwards

        model.eval()

        train_correct = 0
        train_total = 0
        with torch.no_grad():
            for data in tqdm(train_loader):
                images, labels = data
                
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()
                
        #str(epoch) if just starting to train
        #str(epoch+epoch_before+1) afterwards
        print("epoch: " + str(epoch+epoch_before+1))
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - train_correct / train_total)))


        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(val_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                val_total += labels.size(0)

                val_correct += (predicted == labels).sum().item()
        
        #str(epoch) if just starting to train
        #str(epoch+epoch_before+1) afterwards
        print("epoch: " + str(epoch+epoch_before+1))
        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - val_correct / val_total)))
        

        gc.collect()

In [ ]:
train()

## Prediction on Testset

In [ ]:
test_loader = dataloaders['test']
checkpoint = torch.load('INSERT MODEL PATH HERE') #for ex: torch.load('../input/trained-model/model_21.pth')
model.load_state_dict(checkpoint['state_dict'])

model.eval()
fn_list = []
pred_list = []
for x, fn in test_loader:
    x = x.to(device)
    output = model(x)
    pred = torch.argmax(output, dim=1)
    fn_list += fn
    pred_list += [p.item() for p in pred]

submission = pd.DataFrame({"image_name":fn_list, "target":pred_list})
submission.to_csv('prediction.csv', index=False)